Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб.,
после чего выпадает случайное целое число, равномерно распределенное от 0 до 999.
При некоторых значениях числа игрок получает выигрыш (см. справа)

Выгодна ли игра игроку?

Сколько в среднем приобретает
или теряет игрок за одну игру?

- 777: 200 руб.
- 999: 100 руб.
- 555: 50 руб.
- 333: 15 руб.
- 111: 10 руб.
- *77: 5 руб.
- **7: 3 руб.
- *00: 2 руб.
- **0: 1 руб.

* – любая цифра


**Дополнительное задание повышенной сложности.**

Теоретически рассчитать средний выигрыш (проигрыш) и сравнить с результатами моделирования

In [1]:
import numpy as np
import re


In [84]:
iterations = 1000000
gamer_payed = 0
gamer_won = 0

In [79]:
winning_nums =  {
    '777': 200, '999': 100, '555': 50, '333': 15, '111': 10
}

winning_lookups = [
    (r'([012345689]{0,2})(?<!7)77$', 5), # *77
    (r'([012345689]{1,2})(?<!7)7$', 3), # **7
    (r'[0-9]{0,2}00$', 2),  # *00
    (r'[0-9]{0,2}(?<!0)0$', 1)  # **0
]

def gamble():
    global gamer_payed
    global gamer_won

    gamer_payed += 1
    result = ""

    for i in range(3):
        result += str(np.random.randint(0, 10))

    # print(f"Your numbers are: {result}")

    if result in winning_nums:
        gamer_won += winning_nums[result]
        # print(f"Congrats! You won: {winning_nums[result]}. Total: {gamer_won}")
        return gamer_won, gamer_payed
    else:
        for pattern, value in winning_lookups:
            if re.match(pattern, result):
                gamer_won += value
                # print(f"Congrats! You won: {value}. Total: {gamer_won}")
                return gamer_won, gamer_payed

        # print(f'Sorry, you lost. '
        #       f'You spent: {gamer_payed}, you won: {gamer_won}')
    return gamer_won, gamer_payed

### Комментарий к домашней работе от преподавателя:
оказываются неверные выигрыши для комбинаций 000, 007, * 70, 7 * 0

Выигрыш для комбинации:
- 000 = 2 рубля
- 007 = 3 рубля
- *70 = 1 рубль
- 7*0 = 1 рубль

Кроме этого, вы задаёте диапазон случайных чисел до 998, а не 999 - исправлено

Были исправлены регулярные выражения для *00 и **0, регулярные выражения для **7 и *77 - корректны.
В ходе тестирования ошибок обнаружения для 007, * 70, 7 * 0 обнаружено не было.

Был полностью изменен механизм формирования комбинации -
каждая цифра в числе формируется по отдельности последовательно, так же как это происходит в игровом автомате.
Таким образом, теперь могут появляться числовые комбинации состоящие из 0 или начинающиеся на один или два 0.

In [61]:
# Проверка для 000
# Регулярное выражение отрабатывает корректно, проблема была в том, что в переменную result записывалось число 000,
# которое после конвертации в строку преобразовывалось в 0, что и давало некорректный результат.

gamble()

Your numbers are: 000
Congrats! You won: 2. Total: 15


(15, 20)

In [68]:
# Проверка для 007

gamble()

Your numbers are: 007
Congrats! You won: 3. Total: 3


(3, 1)

In [73]:
# Проверка для *70

gamble()


Your numbers are: 170
Congrats! You won: 1. Total: 1


(1, 1)

In [75]:
# Проверка для 7*0

gamble()

Your numbers are: 710
Congrats! You won: 1. Total: 2


(2, 2)

In [85]:
for i in range(iterations):
    gamble()

print(
    f"Игр: {iterations} \n"
    f"Потрачено денег: {gamer_payed} \n"
    f"Выиграно денег: {gamer_won}\n"
    f"Игрок приобретает в среднем за 1 игру: {(gamer_won-gamer_payed)/ iterations} денег"
)

# Игра не выгодна, при увеличении количества интераций средний проигрыш колеблется в районе -0,2 рублей
# При итерациях меньше 1000 игра иногда может быть выгодной

Игр: 1000000 
Потрачено денег: 1000000 
Выиграно денег: 786824
Игрок приобретает в среднем за 1 игру: -0.213176 денег


## Теоретический расчет среднего выигрыша

События:

- P(выигрыша) = количество всех выигрышных комбинаций/ все возможные комбинации
- P(проигрыша) = 1 - P(выигрыша)

- P(любая цифра) = 1 или 100%

- P(конкретная цифра) = 0,1 - 10%, один шанс из 10
- P(все кроме конкретной цифры) = 1- P(конкретная цифра) = 1 - 0,1

- P(&lowast;&lowast;7) = 1&lowast;1&lowast;0,1
- P(&lowast;77) =  = 1&lowast;0,1&lowast;0,1
- P(777)= P(конкретная цифра) &lowast; P(конкретная цифра) &lowast; P(конкретная цифра) = 0,1&lowast;0,1&lowast;0,1

In [4]:
prob_777 = 0.1*0.1*0.1 #вероятность получения 200р
prob_999 = 0.1*0.1*0.1 #вероятность получения 100р
prob_555 = 0.1*0.1*0.1 #вероятность получения 50р
prob_333 = 0.1*0.1*0.1 #вероятность получения 15р
prob_111 = 0.1*0.1*0.1 #вероятность получения 10р
prob_x77 = (1*0.1*0.1) - prob_777 #вероятность получения 5р
prob_xx7 = (1*1*0.1) - prob_777 - prob_x77 #вероятность получения 3р
prob_x00 = 1*0.1*0.1 #вероятность получения 2р
prob_xx0 = (1*1*0.1) - prob_x00 #вероятность получения 1р

prob_winning = (prob_777+ prob_999+ prob_555 + prob_333 +
                prob_111 + prob_x77 + prob_xx7 + prob_x00 + prob_xx0) # вероятность получения выигрыша

average_winning = (
    (prob_777*(200-1))+  # из суммы выигрыша вычитаем затраты на игру
    (prob_999*(100-1))+
    (prob_555*(50-1))+
    (prob_333*(15-1))+
    (prob_111*(10-1))+
    (prob_x77*(5-1))+
    (prob_xx7*(3-1))+
    (prob_x00*(2-1))+
    (prob_xx0*(1-1)) )

average_losing = (1-prob_winning)*(0-1)


print(f"Теоретический чистый средний выигрыш: {average_winning} \n"
      f"Теоретический чистый средний проигрыш: {average_losing} \n"
      f"Выгода: {round(average_winning+average_losing, 3)}")

Теоретический чистый средний выигрыш: 0.5960000000000001 
Теоретический чистый средний проигрыш: -0.796 
Выгода: -0.2


(после исправления)

Теоретический расчет среднего выигрыша имеет практически такое же значение с разельтатами моделирования,
разница в 0,01 - 0,02.
Вывод однозначный - игра не выгодная.
